In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/11/12 20:51:15 WARN Utils: Your hostname, Llama resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/12 20:51:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 20:51:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.4.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [41]:
film_count = film_df.join(film_category_df, on="film_id", how="inner")\
                    .join(category_df, on="category_id", how="inner")\
                    .groupBy(F.column("name"))\
                    .count()\
                    .orderBy(F.column("count").desc())
film_count.show()

+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [22]:
top_10_rental_sum_actors = actor_df.join(film_actor_df, on="actor_id", how="inner")\
                                   .join(film_df, on="film_id", how="inner")\
                                   .join(inventory_df, on="film_id", how="inner")\
                                   .join(rental_df, on="inventory_id", how="inner")\
                                   .join(payment_df, on="rental_id", how="inner")\
                                   .groupBy(actor_df["actor_id"], actor_df["first_name"], actor_df["last_name"])\
                                   .sum("amount")\
                                   .withColumnRenamed("sum(amount)", "sum_amount")\
                                   .orderBy(F.column("sum_amount").desc())\
                                   .limit(10)
top_10_rental_sum_actors.show()

+--------+----------+-----------+------------------+
|actor_id|first_name|  last_name|        sum_amount|
+--------+----------+-----------+------------------+
|     107|      GINA|  DEGENERES|3442.4899999999434|
|     181|   MATTHEW|     CARREY| 2742.189999999965|
|     198|      MARY|     KEITEL| 2689.249999999977|
|      81|  SCARLETT|      DAMON|2655.2799999999743|
|     102|    WALTER|       TORN| 2620.619999999978|
|     144|    ANGELA|WITHERSPOON|2614.4599999999723|
|      58| CHRISTIAN|     AKROYD|2611.4899999999802|
|      60|     HENRY|      BERRY|2602.8799999999746|
|      28|     WOODY|    HOFFMAN|2546.3999999999864|
|     111|   CAMERON|  ZELLWEGER| 2529.409999999984|
+--------+----------+-----------+------------------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [30]:
most_money_per_category = film_df.join(film_category_df, on="film_id", how="inner")\
                                 .join(category_df, on="category_id", how="inner")\
                                 .join(inventory_df, on="film_id", how="inner")\
                                 .join(rental_df, on="inventory_id", how="inner")\
                                 .join(payment_df, on="rental_id", how="inner")\
                                 .groupBy(category_df["category_id"], category_df["name"])\
                                 .sum("amount")\
                                 .withColumnRenamed("sum(amount)", "sum_amount")\
                                 .orderBy(F.column("sum_amount").desc())\
                                 .limit(1)
most_money_per_category.show()

+-----------+------+-----------------+
|category_id|  name|       sum_amount|
+-----------+------+-----------------+
|         15|Sports|5314.209999999843|
+-----------+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [34]:
films_no_inventory = film_df.join(inventory_df, on="film_id", how="left")\
                            .where(F.column("inventory_id").isNull())\
                            .select("title")
films_no_inventory.show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [40]:
top_3_children_film_actors = actor_df.join(film_actor_df, on="actor_id", how="inner")\
                                     .join(film_df, on="film_id", how="inner")\
                                     .join(film_category_df, on="film_id", how="inner")\
                                     .join(category_df, on="category_id", how="inner")\
                                     .filter(F.column("name") == "Children")\
                                     .groupBy(F.column("actor_id"), F.column("first_name"), F.column("last_name"))\
                                     .count()\
                                     .orderBy(F.column("count").desc())\
                                     .select(F.column("first_name"), F.column("last_name"), F.column("count"))\
                                     .limit(3)
top_3_children_film_actors.show()

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|     HELEN|   VOIGHT|    7|
|     KEVIN|  GARLAND|    5|
|    WHOOPI|     HURT|    5|
+----------+---------+-----+



Stop Spark session:

In [42]:
spark.stop()